# ESS Data Usage Example - MTMount accelerometers
Querying MTMount accelerometers \
Craig Lage - 22-Mar-23

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import time
import warnings

from astropy.time import Time, TimeDelta
from lsst_efd_client.efd_helper import EfdClient, merge_packed_time_series

# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
# Get EFD client
client = EfdClient("usdf_efd")

In [ ]:
# Times to look at the data
# start = Time("2023-03-22 08:00:00Z", scale="utc")
# end = Time("2023-03-22 08:10:00Z", scale="utc")
# start = Time("2024-03-08 08:00:00Z", scale="utc")
# end = Time("2024-03-08 08:10:00Z", scale="utc")

start = Time("2024-03-27 09:00:00Z", scale="utc")
end = Time("2024-03-27 09:05:00Z", scale="utc")

In [ ]:
baseFields = ["accelerationX", "accelerationY", "accelerationZ"]
sensorNames = [
    "SST top end ring +x -y",
    "SST top end ring -x -y",
    "SST spider spindle",
    "SST M2 surrogate",
]

##  Get all of the data for the selected times

In [ ]:
packed_dataframe = await client.select_time_series(
    "lsst.sal.ESS.accelerometer", ["*"], start, end
)

## Now unpack it and plot it by sensor

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for i, sensorName in enumerate(sensorNames):
    sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
    plotx = i % 2
    ploty = int(i / 2)
    ax = axs[plotx][ploty]
    ax.set_title(sensorName)
    ax.set_ylabel("Accel(m/s^2)")
    for baseField in baseFields:
        df = merge_packed_time_series(
            sub_dataframe,
            baseField,
            stride=1,
            ref_timestamp_col="timestamp",
            fmt="unix_tai",
            scale="tai",
        )
        df[baseField].plot(ax=ax, label=list(baseField)[-1])
    ax.legend()
plt.savefig("/home/c/cslage/u/MTMount/vel_accel_jerk_plots/Accel_Test_20240327.png")